In [54]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [55]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'data/train.csv.zip' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

# Data cleaning and normalisation

In [56]:
# we know that very low values are used to signal unavailable data
tX[tX < -900] = np.nan

In [57]:
from proj1_helpers import sample_data, standardize
seed = 1

tX_mean = np.nanmean(tX,axis=0)

tX_std = np.nanstd(tX,axis=0)
norm_tX = np.subtract(tX, tX_mean, where=np.isfinite(tX_mean))
norm_tX = np.divide(norm_tX, tX_std, where=tX_std>0)
norm_tX[np.isnan(norm_tX)] =0
#norm_tX = np.c_[np.ones(num_samples), norm_tX]

#y, norm_tX = sample_data(y, norm_tX, seed, len(y[0]))#size_samples= 100000)
norm_tX, mean_x, std_x = standardize(norm_tX)

norm_tX = np.c_[np.ones((y.shape[0], 1)), norm_tX]
initial_w = np.zeros((norm_tX.shape[1], 1))


#print(initial_w)



# Logistic Regression


In [61]:
from Implementation import logistic_regression_gd

# init parameters
max_iter = 1000
gamma = 0.00000009
initial_w = np.zeros((norm_tX.shape[1], 1))

#print(np.shape(y))
#print(np.shape(norm_tX))
#print(np.shape(initial_w))

    
weights = logistic_regression_gd(y, norm_tX, initial_w, max_iter, gamma)



Current iteration=0, loss=173286.79513997526, gamma=9e-08, gradient = [[ 2.03666000e+05]
 [-2.60878135e+03]
 [ 8.33941282e+04]
 [ 3.33532744e+03]
 [-4.56866480e+04]
 [-4.39684422e+04]
 [-4.24177513e+04]
 [ 3.94195876e+04]
 [-2.90586227e+03]
 [ 3.62771829e+03]
 [-3.63630064e+04]
 [ 4.63680733e+04]
 [-6.44869325e+04]
 [-4.12599488e+04]
 [-5.58221553e+04]
 [ 2.23834212e+02]
 [ 1.04472587e+03]
 [ 7.58118729e+03]
 [-3.59803838e+02]
 [-9.78971891e+02]
 [-5.33114037e+03]
 [-1.77390454e+03]
 [-3.21590638e+04]
 [-3.16913111e+04]
 [-2.07761451e+04]
 [-5.84093585e+01]
 [ 4.19142831e+01]
 [ 3.02851427e+03]
 [-1.61418076e+02]
 [ 8.42878944e+02]
 [-3.18684808e+04]]
Current iteration=100, loss=12086.40984498414, gamma=1e-09, gradient = [[ 1.71098964e+05]
 [-1.91337345e+03]
 [ 6.02670888e+04]
 [ 1.69275375e+03]
 [-2.11288406e+04]
 [-2.57547350e+04]
 [-2.33638437e+04]
 [ 2.14867000e+04]
 [-1.41530162e+04]
 [ 9.12972605e+03]
 [-1.29102598e+04]
 [ 3.28292005e+04]
 [-4.05026176e+04]
 [-2.56713685e+04]
 [-

KeyboardInterrupt: 

# Regularized logistic regression


In [62]:
from Implementation import regu_logistic_regression_gd

# init parameters
max_iter = 1000
gamma = 0.00000009
lambda_ = 0


initial_w = np.zeros((norm_tX.shape[1], 1))


weights = regu_logistic_regression_gd(y, norm_tX, lambda_, initial_w, max_iter, gamma )

Current iteration=0, loss=173286.79513997526, gradient = [[ 2.03666000e+05]
 [-2.60878135e+03]
 [ 8.33941282e+04]
 [ 3.33532744e+03]
 [-4.56866480e+04]
 [-4.39684422e+04]
 [-4.24177513e+04]
 [ 3.94195876e+04]
 [-2.90586227e+03]
 [ 3.62771829e+03]
 [-3.63630064e+04]
 [ 4.63680733e+04]
 [-6.44869325e+04]
 [-4.12599488e+04]
 [-5.58221553e+04]
 [ 2.23834212e+02]
 [ 1.04472587e+03]
 [ 7.58118729e+03]
 [-3.59803838e+02]
 [-9.78971891e+02]
 [-5.33114037e+03]
 [-1.77390454e+03]
 [-3.21590638e+04]
 [-3.16913111e+04]
 [-2.07761451e+04]
 [-5.84093585e+01]
 [ 4.19142831e+01]
 [ 3.02851427e+03]
 [-1.61418076e+02]
 [ 8.42878944e+02]
 [-3.18684808e+04]]
Current iteration=100, loss=-175304.56711818115, gradient = [[ 1.39677497e+05]
 [-5.83959913e+02]
 [ 5.04079337e+04]
 [ 1.86045022e+03]
 [-1.33748162e+04]
 [-1.40708371e+04]
 [-1.33839710e+04]
 [ 1.09985920e+04]
 [-1.71244524e+04]
 [ 8.29453934e+03]
 [-5.86010883e+03]
 [ 2.60617448e+04]
 [-3.05210295e+04]
 [-1.59503414e+04]
 [-2.61824991e+04]
 [ 2.576

KeyboardInterrupt: 

# Least squares and Polynomial regression


In [168]:
def least_squares(y, tx):
    """calculate the least squares solution."""
    a = tx.T.dot(tx)
    b = tx.T.dot(y)
    
    
    return np.linalg.solve(a, b)

In [169]:
from costs import compute_mse
from plots import *

def polynomial_regression(x):
    """Constructing the polynomial basis function expansion of the data,
       and then running least squares regression."""
    # define parameters
    degrees = [1,3,7,12]
      
    for ind, degree in enumerate(degrees):
        # form dataset to do polynomial regression.
        tx = build_poly(x, degree)
        
        # least squares
        weights = least_squares(y, tx)
             
        # compute RMSE
        rmse = np.sqrt(2 * compute_mse(y, tx, weights))
        print("Processing {i}th experiment, degree={d}, rmse={loss}".format(
              i=ind + 1, d=degree, loss=rmse))
       
        # print
        #print(np.shape(y))
        #print(np.shape(tx))
        #print(np.shape(weights))
        
    return weights

In [170]:
#weights = polynomial_regression(norm_tX)
#weights = polynomial_regression(norm_tX2)

In [171]:
def split_data(x, y, ratio, seed=1):
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(seed)
    # generate random indices
    num_row = len(y)
    indices = np.random.permutation(num_row)
    index_split = int(np.floor(ratio * num_row))
    index_tr = indices[: index_split]
    index_te = indices[index_split:]
    # create split
    x_tr = x[index_tr]
    x_te = x[index_te]
    y_tr = y[index_tr]
    y_te = y[index_te]
    return x_tr, x_te, y_tr, y_te

In [172]:
def train_test_split_demo(x, y, degree, ratio, seed):
    """polynomial regression with different split ratios and different degrees."""
    x_tr, x_te, y_tr, y_te = split_data(x, y, ratio, seed)
    # form tx
    tx_tr = build_poly(x_tr, degree)
    tx_te = build_poly(x_te, degree)

    weight = least_squares(y_tr, tx_tr)
    
    #if ((ratio == 0,9) and (degree == 7)):
    weights = weight
    #print(weights)

    # calculate RMSE for train and test data.
    rmse_tr = np.sqrt(2 * compute_mse(y_tr, tx_tr, weight))
    rmse_te = np.sqrt(2 * compute_mse(y_te, tx_te, weight))

    print("proportion={p}, degree={d}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
          p=ratio, d=degree, tr=rmse_tr, te=rmse_te))
    return weight

In [173]:
def train_test_split_demo_2(x, y, degree, ratio, seed, w):
    """polynomial regression with different split ratios and different degrees."""
    x_tr, x_te, y_tr, y_te = split_data(x, y, ratio, seed)
    # form tx
    tx_tr = build_poly(x_tr, degree)
    tx_te = build_poly(x_te, degree)

    weight = w
    
    # calculate RMSE for train and test data.
    rmse_tr = np.sqrt(2 * compute_mse(y_tr, tx_tr, weight))
    rmse_te = np.sqrt(2 * compute_mse(y_te, tx_te, weight))

    print("proportion={p}, degree={d}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
          p=ratio, d=degree, tr=rmse_tr, te=rmse_te))
    return weight
    
    #if ((ratio == 0,9) and (degree == 7)):
    
    #print(weights)

    # calculate RMSE for train and test data.
    rmse_tr = np.sqrt(2 * compute_mse(y_tr, tx_tr, weight))
    rmse_te = np.sqrt(2 * compute_mse(y_te, tx_te, weight))

    print("proportion={p}, degree={d}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
          p=ratio, d=degree, tr=rmse_tr, te=rmse_te))
    return weight

In [174]:
seed = 6
degrees = [7]
split_ratios = [0.9]
#weights = train_test_split_demo_2(norm_tX, y, degree, split_ratio, seed, weights)
#weights = train_test_split_demo(norm_tX, y, degree, split_ratio, seed)
#print(weights)

#for split_ratio in split_ratios:
    #for degree in degrees:
      #  train_test_split_demo(norm_tX, y, degree, split_ratio, seed)
        


## Generate predictions and save ouput in csv format for submission:

In [175]:
DATA_TEST_PATH = 'data/test.csv.zip' # TODO: download train data and supply path here 
y, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [176]:
# we know that very low values are used to signal unavailable data
#tX_test[tX_test < -900] = np.nan

In [177]:
tX_test_mean = np.nanmean(tX_test,axis=0)
tX_test_std = np.nanstd(tX_test,axis=0)
norm_tX_test = np.subtract(tX_test, tX_test_mean, where=np.isfinite(tX_test_mean))
norm_tX_test = np.divide(norm_tX_test, tX_test_std, where=tX_test_std>0)
norm_tX_test = np.c_[np.ones((y.shape[0], 1)), norm_tX_test]
#norm_tX_test = build_poly(norm_tX_test, 7)


In [178]:
OUTPUT_PATH = 'data/sample-submission' # TODO: fill in desired name of output file for submission
#print(np.shape(y))
#print(np.shape(norm_tX_test))
#print(np.shape(weights))
print(weights)
y_pred = predict_labels(weights, norm_tX_test)

create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

[[-6.18512471e-01]
 [ 7.54498170e-03]
 [-2.31343383e-01]
 [-7.36392543e-03]
 [ 1.00090770e-01]
 [ 1.10259604e-01]
 [ 1.02141798e-01]
 [-9.52617251e-02]
 [ 3.43406399e-02]
 [-2.48680395e-02]
 [ 7.10424253e-02]
 [-1.27639651e-01]
 [ 1.65819248e-01]
 [ 1.06507895e-01]
 [ 1.44645602e-01]
 [-8.68444579e-04]
 [-3.22895969e-03]
 [-2.31962345e-02]
 [ 5.67026833e-04]
 [ 2.89662835e-03]
 [-4.37065458e-03]
 [ 4.60402877e-03]
 [ 5.90859737e-02]
 [ 6.28703060e-02]
 [ 3.36388848e-02]
 [ 1.03863673e-04]
 [-4.23042045e-05]
 [-2.42682110e-02]
 [ 6.29620467e-04]
 [-2.08265797e-03]
 [ 5.60120526e-02]]
[[-1.62337415]
 [-1.14891405]
 [-1.15359958]
 ...
 [-0.68263614]
 [-0.30230101]
 [-1.5155485 ]]
